# Postprocess predictions and create a submission

In [1]:
#@title Submission notes and params { run: "auto" }
version_notes = "BBox reduction" #@param {type:"string"}

reduce_method = "weighted_boxes_fusion" #@param ["None", "nms", "soft_nms", "non_maximum_weighted", "weighted_boxes_fusion"]

iou_threshold = 0.6 #@param {type:"slider", min:0, max:1, step:0.01}
skip_box_threshold = 0.01 #@param {type:"slider", min:0, max:1, step:0.01}
sigma = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

binary_abnormal_threshold = 0.05 #@param {type:"slider", min:0, max:1, step:0.01}
binary_normal_threshold = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}
assert(binary_abnormal_threshold <= binary_normal_threshold)

SUBMIT = True #@param {type:"boolean"}

In [2]:
#@title Select models versions { run: "auto" }

binary_version = "7: EfficientNetB3, include probabilities (for later double thresholding); image size: (640, 640); threshold: 0.8" #@param ["2: Try 1024 resolution", "3: EffNet, 1024 resolution, threshold 0.8", "4: EffNet, 1024 resolution, threshold 0.8", "5: EffNet, 512 resolution, threshold 0.8", "6: EfficientNetB3, include probabilities (for later double thresholding); image size: (1024, 1024); threshold: 0.8", "7: EfficientNetB3, include probabilities (for later double thresholding); image size: (640, 640); threshold: 0.8"]
binary_notes = binary_version.split(":")[1]
binary_version = int(binary_version.split(":")[0])


yolo_versions_include = "33, 34, 35, 36, 37" #@param {type:"string"}
yolo_versions_include = list(map(int, yolo_versions_include.split(","))) if yolo_versions_include else []
print(yolo_versions_include)

[33, 34, 35, 36, 37]


In [3]:
INPUTS_YOLO_VERSIONS = {
    5: {
        "version_notes": "YOLOv5x, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15",
        # "path": f"v{version}",
        # "enabled": version in yolo_versions_include,
    },
    6: { "version_notes": "WBF preproc, YOLOv5x, 50 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15", },
    7: { "version_notes": "WBF preproc, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1", },
    8: { "version_notes": "WBF preproc with IoU 0.4, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15, remove (most) augs", },
    9: { "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs", },
    10: { "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 25 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs", },
    11: { "version_notes": "Remove (most) augs --- WBF preproc IoU: 0.8, version: yolov5l, epochs: 30, valid split: 20.0%, image size: 1024, detect IoU: 0.8, detect conf: 0.05", },
    # 5 Folds of the same model
    12: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 0/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    13: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 1/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    14: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 2/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    15: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 3/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    16: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 4/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    # version 17 and 18 are identical.
    # 5 Folds, WBF at 0.4
    18: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: exp_yolov5x_50epochs_WBFat0.4iou_640image_detect0.6iou0.01conf_fold0of5", },
    19: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    20: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    21: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    22: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    #
    23: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.65; detect conf: 0.001; exp: yolov5x_40epochs_640image_fold0of5_WBFat0.4iou_det0.65iou0.001conf", },
    24: { "version_notes": "Rad weight as number of labels, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_50epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    # 5 Folds for outliers filter with Gaussian model
    25: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    26: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    27: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    28: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    29: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    # version 30 did very poorly in CV
    31: { "version_notes": "Filter outliers (bbox size) with quantiles, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.8; detect conf: 0.01; exp: yolov5x_outliers_quantiles_40epochs_640image_fold0of5_WBFat0.4iou_det0.8iou0.01conf", },
    32: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 100; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gss_100epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    # versions 33-37 are versions 25-29 with TTA
    33: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    34: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    35: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    36: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    37: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    #
    38: { "version_notes": "Filter outliers (bbox size) with Gaussian model, use focal loss at 1.5, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 60; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gss_fl_60epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
}

# Check that versions are distinct (no need to ensemble the same model)
# The dictionary should allow duplicate keys anyway.
assert(len(set(INPUTS_YOLO_VERSIONS.keys())) == len(INPUTS_YOLO_VERSIONS))

for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    # Check for errors in fields.
    assert(all(key_name in version_data for key_name in ["version_notes"]))

    version_data["path"] = f"v{yolo_version}"
    version_data["enabled"] = yolo_version in yolo_versions_include

# Filter only enabled.
INPUTS_YOLO_VERSIONS = {yolo_version: version_data for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items() if version_data["enabled"]}

## Setup

In [4]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/Colab\ Notebooks/kaggle
    from setup_colab import setup_colab_for_kaggle, WORK_FOLDER
    setup_colab_for_kaggle(check_env=False, local_working=True)
except:
    print("Not in Colab")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Content of Drive Kaggle data dir (/content/drive/MyDrive/kaggle): ['/content/drive/MyDrive/kaggle/input', '/content/drive/MyDrive/kaggle/working', '/content/drive/MyDrive/kaggle/.ipynb_checkpoints', '/content/drive/MyDrive/kaggle/output']
Content of Kaggle data dir (/kaggle): ['/kaggle/output', '/kaggle/working', '/kaggle/input']
Content of Kaggle data subdir (/kaggle/input): ['/kaggle/input/cassava-model', '/kaggle/input/cassava-leaf-disease-classification', '/kaggle/input/googlebitemperedloss', '/kaggle/input/vbdyolo', '/kaggle/input/.ipynb_checkpoints', '/kaggle/input/vinbigdata', '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection', '/kaggle/input/vinbigdata-chest-xray-origi

In [5]:
from pathlib import Path

from IPython.display import clear_output

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

INPUT_FOLDER_ORIGINAL_PNG = WORK_FOLDER / "vinbigdata-chest-xray-original-png"
INPUT_FOLDER_YOLO_OUT = WORK_FOLDER / "vbdyolo-out"
INPUT_FOLDER_BINARY = WORK_FOLDER / "vbdbinary"

for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    version_data["path"] = INPUT_FOLDER_YOLO_OUT / version_data["path"]

print(f"Using the following YOLO versions:")
print("\n".join(f"{yolo_version}: {version_data}" for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items()))

Using the following YOLO versions:
33: {'version_notes': 'TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf', 'path': PosixPath('/kaggle/working/vbdyolo-out/v33'), 'enabled': True}
34: {'version_notes': 'TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf', 'path': PosixPath('/kaggle/working/vbdyolo-out/v34'), 'enabled': True}
35: {'version_notes': 'TTA ++ Filter outliers (bbox size) with Gaussi

## Get data from Kaggle

In [6]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@fix-datasets-download-file-unzip
clear_output()

In [7]:
!kaggle datasets download corochann/vinbigdata-chest-xray-original-png -f test_meta.csv -p {INPUT_FOLDER_ORIGINAL_PNG} --unzip

  0% 0.00/126k [00:00<?, ?B/s]
100% 126k/126k [00:00<00:00, 55.0MB/s]


In [8]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@add-datasets-download-version
clear_output()

In [9]:
for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    !rm -rf {version_data["path"]}
    !kaggle datasets download "witalia/vbdyolo-out-newest" -v {yolo_version} -p {version_data["path"]} --unzip --force

 99% 307M/311M [00:02<00:00, 102MB/s]
100% 311M/311M [00:02<00:00, 114MB/s]
 93% 290M/311M [00:02<00:00, 83.9MB/s]
100% 311M/311M [00:02<00:00, 118MB/s] 
 93% 290M/311M [00:02<00:00, 80.1MB/s]
100% 311M/311M [00:03<00:00, 107MB/s] 
 93% 289M/311M [00:03<00:00, 78.4MB/s]
100% 311M/311M [00:03<00:00, 105MB/s] 
 93% 289M/311M [00:02<00:00, 121MB/s]
100% 311M/311M [00:02<00:00, 115MB/s]


In [10]:
!rm -rf {INPUT_FOLDER_BINARY}
!kaggle datasets download "witalia/vbdbinary" -v {binary_version} -p {INPUT_FOLDER_BINARY} --unzip --force

 84% 89.0M/106M [00:01<00:00, 51.8MB/s]
100% 106M/106M [00:01<00:00, 84.4MB/s] 


## Process YOLO output

### BBox reduction (potentially from multiple models)

In [11]:
!pip install ensemble-boxes
clear_output()

In [12]:
from ensemble_boxes import *
from typing import List

def reduce_boxes(image_labels_list: List[pd.DataFrame], iou_thr=iou_threshold, skip_box_thr=skip_box_threshold, sigma=sigma, method=None):
    label_empty = [len(df) == 0 for df in image_labels_list]

    if len(image_labels_list) == 0 or all(label_empty):
        # If there's nothing to fuse, return empty.
        return pd.DataFrame(columns=["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    # Ensemble_boxes doesn't do well with empty inputs, so skip those.
    labels = [image_labels["class_id"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    scores = [image_labels["conf"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    boxes = [image_labels[["x_min", "y_min", "x_max", "y_max"]].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]

    if method == non_maximum_weighted or method == weighted_boxes_fusion:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    elif method == soft_nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, thresh=skip_box_thr, sigma=sigma)
    elif method == nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr)
    else:
        if len(labels) > 1:
            raise ValueError("Cannot fuse more than 1 model without a fusing method")
        [boxes], [scores], [labels] = boxes, scores, labels

    reduced_labels = pd.DataFrame().reindex(list(range(len(labels))))
    reduced_labels["class_id"] = np.array(labels).astype(np.int32)
    reduced_labels["conf"] = scores
    reduced_labels[["x_min", "y_min", "x_max", "y_max"]] = boxes

    return reduced_labels


def reduce_max_one_class(image_labels: pd.DataFrame):
    # Aortic Enlargement and Cardiomegaly should be max one on the image.
    one_class_mask = image_labels["class_id"].isin([0, 3])

    result_df = image_labels[~one_class_mask]
    # Take max 2 most confident boxes in each class.
    result_df_one_class = image_labels[one_class_mask].sort_values("conf", ascending=False).groupby("class_id").head(2)
    result_df = result_df.append(result_df_one_class)

    return result_df

### Read and transform labels

In [13]:
def read_prediction_labels(filename: Path):
    yolo_columns = ["class_id", "x_centre", "y_centre", "bw", "bh", "conf"]
    if filename.exists():
        labels: pd.DataFrame = pd.read_csv(filename, delimiter=" ", header=None)
        labels.columns = yolo_columns
    else:
        labels: pd.DataFrame = pd.DataFrame(columns=yolo_columns)

    # Convert YOLO format (x_centre, y_centre, bw, bh) to competition format (x_min, y_min, x_max, y_max)
    labels["x_min"] = (labels["x_centre"] - labels["bw"] / 2).clip(0, 1)
    labels["y_min"] = (labels["y_centre"] - labels["bh"] / 2).clip(0, 1)
    labels["x_max"] = (labels["x_centre"] + labels["bw"] / 2).clip(0, 1)
    labels["y_max"] = (labels["y_centre"] + labels["bh"] / 2).clip(0, 1)
    labels = labels.drop(columns=["x_centre", "y_centre", "bw", "bh"])
    # After dropping, conf column should become the second one.
    assert(labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    return labels


def scale_labels(labels: pd.DataFrame, image_w: int, image_h: int):
    # Scale coordinates to image's size. Clip to make sure it's not out of bounds of the image.
    labels[["x_min", "x_max"]] = (labels[["x_min", "x_max"]] * image_w).round().astype(np.int32).clip(0, image_w - 1)
    labels[["y_min", "y_max"]] = (labels[["y_min", "y_max"]] * image_h).round().astype(np.int32).clip(0, image_h - 1)

    return labels


def labels_to_string(labels: pd.DataFrame):
    # Empty DataFrame means there was no YOLO output file or "reduce" removed all bboxes.
    # Means "No finding"
    if len(labels) == 0:
        return "14 1 0 0 1 1"
    # Convert all rows to one prediction string
    return " ".join(labels.to_string(header=False, index=False).split())

### Process YOLO output files, and calculate the predictions

In [14]:
import ensemble_boxes

results_yolo_df = pd.DataFrame(columns=["image_id", "PredictionString"])

test_metadata = pd.read_csv(INPUT_FOLDER_ORIGINAL_PNG / "test_meta.csv")
test_metadata = test_metadata.set_index("image_id").to_dict("index")

for image_id, image_dims in tqdm(test_metadata.items(), total=len(test_metadata)):
    # Read YOLO output to pandas DataFrame.
    prediction_labels = [read_prediction_labels(
        version_data["path"] / "labels_pred" / f"{image_id}.txt"
    ) for version_data in INPUTS_YOLO_VERSIONS.values()]

    # Reduce bboxes.
    method = None if reduce_method == "None" else getattr(ensemble_boxes, reduce_method)
    prediction_labels = reduce_boxes(prediction_labels, method=method)

    # Reduce those that should only have one class
    # prediction_labels = reduce_max_one_class(prediction_labels)

    # Scale them to the original size.
    # NOTE: dim0 and dim1 are reversed: y-axis, x-axis!
    prediction_labels = scale_labels(prediction_labels, image_dims["dim1"], image_dims["dim0"])

    # Convert resulting bboxes from DataFrame to string.
    prediction_str = labels_to_string(prediction_labels)

    results_yolo_df = results_yolo_df.append({"image_id": image_id, "PredictionString": prediction_str}, ignore_index=True)

results_yolo_df.sample(10, random_state=10)

,image_id,PredictionString
1779,9b41c13ac918adcc1261f750a1d61b40,11 0.396240 674 461 861 541 8 0.328821 421 176...
341,202dc97c4b3ec7e50f4813456938500a,3 0.909277 701 1218 1556 1531 0 0.870117 958 6...
1276,711dc542c6c7e2b21889838cd050a810,7 0.182416 1318 971 1780 1625 7 0.156335 446 1...
1012,5ba0a669fd3430e9386758e838210b98,0 0.493581 1785 1179 2073 1484 7 0.371436 904 ...
470,2b2810284313c5547241b38668f67b2c,3 0.559717 750 1716 1593 2044 0 0.545567 1075 ...
1755,990b0dcd83353894c4a2e73738cd6c25,3 0.356242 1055 1224 2141 1692 5 0.200750 1555...
2773,ec6c4e81b448275577d3e4a74acbe443,0 0.851562 1096 691 1318 945 3 0.605664 847 13...
1085,6100bc75e0a668c382d92d5eb9038076,3 0.683126 668 1098 1547 1404 0 0.400244 1023 ...
2055,b1a596318e7bfdcbb74044bf6d38c6ed,0 0.760352 960 588 1155 793 3 0.688770 638 125...
2585,de7aa2b6adfb9db39489c019f5f3a1a1,3 0.885254 674 1376 1812 1741 0 0.865234 1040 ...


## Merge with Binary classifier output

In [15]:
results_binary_df = pd.read_csv(INPUT_FOLDER_BINARY / "prediction.csv")
display(results_binary_df.head())

results_df = results_yolo_df.merge(results_binary_df, on="image_id")

if "confidence" in results_df.columns:
    # If confidence of "No finding" is very high, replace with "No finding"
    results_df.loc[results_df["confidence"] >= binary_normal_threshold, "PredictionString"] = "14 1 0 0 1 1"
    # If confidence of "No finding" is medium, add "No finding" with confidence to the object detector's bboxes.
    # Skip those that are already "No finding".
    results_df.loc[
            (results_df["confidence"] < binary_normal_threshold) & \
            (results_df["confidence"] > binary_abnormal_threshold) & \
            (~(results_df["PredictionString"] == "14 1 0 0 1 1")),
        "PredictionString"] += results_df["confidence"].apply(lambda prob: f" 14 {prob:.6f} 0 0 1 1")
    # If confidence of "No finding" is low, i.e. confidence of abnormality is high, do nothing.
else:
    # Just use binary classifier's suggestion as to which to set as "No finding"
    results_df.loc[results_df["class_name"] == "normal", "PredictionString"] = "14 1 0 0 1 1"

results_df = results_df[["image_id", "PredictionString"]]

results_df.to_csv(WORK_FOLDER / "submission.csv", index=False)
display(results_df.head())

,image_id,class_name,confidence
0,980141b50ada8a624c9a1c34c91c95d8,abnormal,0.470578
1,f362fd880b0149646ccbb760be5f6354,normal,0.994644
2,0291515f5d14c34180a15712a55bf7bd,abnormal,0.089130
3,6cf8916303a07f3909297be170f21f7a,normal,0.970541
4,c6829051ecb281656c987fa2cfe5c706,abnormal,0.167964


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,0 0.788965 1262 588 1531 889 0 0.117749 1111 5...
2,008bdde2af2462e86fd373a445d0f4cd,3 0.867285 1093 1411 1935 1787 0 0.859864 1431...
3,009bc039326338823ca3aa84381f17f1,3 0.840527 666 1054 1556 1348 0 0.566989 994 4...
4,00a2145de1886cb9eb88869c85d74080,3 0.899316 779 1284 1867 1640 0 0.803516 1120 ...


## Submission overview

Total number of "No finding" before binary classifier filter

In [16]:
print((results_yolo_df["PredictionString"] == "14 1 0 0 1 1").sum())

0


Total number of "No finding"

In [17]:
(results_df["PredictionString"] == "14 1 0 0 1 1").sum()

1161

Total with class "No finding" appended

In [18]:
print(results_df["PredictionString"].str.contains(r" 14 \d\.\d+ 0 0 1 1").sum())

1728


## Submit to Kaggle

In [19]:
submission_message = " ***** ".join([
    version_notes,
    "; ".join([
        f"Reduce method: {reduce_method}",
        f"IoU {iou_threshold}",
        f"Conf {skip_box_threshold}",
        f"Sigma {sigma}",
        f"Binary Normal Thresh: {binary_normal_threshold}",
        f"Binary Abnormal Thresh: {binary_abnormal_threshold}",
    ]),
    f"Binary CLF: {binary_version} -{binary_notes}",
    "Ensemble YOLO: " + "; ".join(f"{yolo_version} - {version_data['version_notes']}" for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items()),
])
!echo "{submission_message}"

if SUBMIT:
    !kaggle competitions submit \
        vinbigdata-chest-xray-abnormalities-detection \
        -f {WORK_FOLDER}/submission.csv \
        -m "{submission_message}"
    !sleep 10
    !kaggle competitions submissions vinbigdata-chest-xray-abnormalities-detection

BBox reduction ***** Reduce method: weighted_boxes_fusion; IoU 0.6; Conf 0.01; Sigma 0.1; Binary Normal Thresh: 0.95; Binary Abnormal Thresh: 0.05 ***** Binary CLF: 7 - EfficientNetB3, include probabilities (for later double thresholding); image size ***** Ensemble YOLO: 33 - TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf; 34 - TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf; 35 - TTA ++ 